An example of strategy based on the markowitz min variance portfolio. Your duty is to propose a particular strategy similar to the one located at the folder 'example' 

Load Data

In [1]:
from modules.load_data import load_data
dict_data = load_data()

A single day example

In [ ]:
from example.strategy_minRisk import strategy_minRisk
strategy_minRisk(dict_data, 500, 100)

Simulator

In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from example.strategy_minRisk import strategy_minRisk
from modules.strategy_simulator import strategy_simulator

# Set the path to the strategy and the strategy itself
path = "example/"  # Path to your strategy
strategy = strategy_minRisk  # Your strategy

# Initialize data structures to store results
ret_port = pd.Series(dtype="float64")  # Series to store portfolio returns
weights_db = pd.DataFrame()  # DataFrame to store historical weights

# Loop through a range of time values
for t in tqdm(range(500, len(dict_data['prices'].index) - 1)):
    # Use the strategy simulator to get portfolio's historical weights [weights_db]
    # and its next day returns [ret_port]
    ret_port, weights_db = strategy_simulator(path, strategy, dict_data, t, ret_port, weights_db)
